In [1]:
import pandas as pd
from json import loads, dumps


In [2]:
datas = ['darmstadt_unis', 'mpqa', 'opener_en']

df1 = [pd.read_json(f"../{i}/train_convert.json") for i in datas]
df = pd.concat(df1)
df.reset_index(drop=True)

,sent_id,raw_words,words,aspects,holder,opinions
0,Colorado_Technical_University_Online_2_06-11-2...,NA I am a current student of their MBA program...,"[NA, I, am, a, current, student, of, their, MB...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]"
1,Colorado_Technical_University_Online_2_06-11-2...,NA I have been in the IT industry for 20 years...,"[NA, I, have, been, in, the, IT, industry, for...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]"
2,Colorado_Technical_University_Online_2_06-11-2...,"NA I am currently a director , and do make a s...","[NA, I, am, currently, a, director, ,, and, do...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]"
3,Colorado_Technical_University_Online_2_06-11-2...,"NA I did my research , and am concerned about ...","[NA, I, did, my, research, ,, and, am, concern...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]"
4,Colorado_Technical_University_Online_2_06-11-2...,"NA Especially , since I am planning for a PhD ...","[NA, Especially, ,, since, I, am, planning, fo...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]"
...,...,...,...,...,...,...
6132,opener_en/kaf/hotel/english00011_0b57ac0ca8569...,NA Golf courses can be easily reached from the...,"[NA, Golf, courses, can, be, easily, reached, ...","[{'index': 0, 'from': 11, 'to': 13, 'term': 't...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'...","[{'index': 0, 'from': 17, 'to': 18, 'term': 'o..."
6133,opener_en/kaf/hotel/english00011_0b57ac0ca8569...,"NA Hotel offers nice spa , large pool area and...","[NA, Hotel, offers, nice, spa, ,, large, pool,...","[{'index': 0, 'from': 1, 'to': 2, 'term': 'Hot...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'...","[{'index': 0, 'from': 2, 'to': 3, 'term': 'off..."
6134,opener_en/kaf/hotel/english00011_0b57ac0ca8569...,NA Hospitality and services at the highest lev...,"[NA, Hospitality, and, services, at, the, high...","[{'index': 0, 'from': 1, 'to': 2, 'term': 'Hos...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'...","[{'index': 0, 'from': 4, 'to': 8, 'term': 'at ..."
6135,opener_en/kaf/hotel/english00011_0b57ac0ca8569...,NA I will definitely come back !,"[NA, I, will, definitely, come, back, !]","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'...","[{'index': 0, 'from': 1, 'to': 2, 'term': 'I'}]","[{'index': 0, 'from': 2, 'to': 7, 'term': 'wil..."


In [3]:
df.to_json(r'ddx.json', orient='records')

In [2]:
from tqdm import tqdm
import os, json

NULL_TOKEN = 'NA'
START_WORD = 'NA '
NULL_INDEX = 0

def char2word_offset(sent, start_end_str):
    start, end = [int(i) for i in start_end_str.split(':')]
    start, end = start+len(START_WORD), end+len(START_WORD)
    word_start, word_end = None, None
    space_count = 0
    for i in range(len(sent)):
        if sent[i]==' ': space_count += 1
        if i>=start and word_start is None: word_start = space_count
        if i>=end and word_start is not None: word_end = space_count; break
    if word_end is None and word_start is not None: word_end = space_count+1
    assert (type(word_start) == type(word_end))
    if word_start is None: word_start, word_end = NULL_INDEX, NULL_INDEX+1
    return int(word_start), int(word_end)

def format_convert(original_data, file, threshold=0):
    final_data = []
    blank_cnt = 0
    threshold *= len(original_data)
    if 't1' in file: print(f'{threshold} limit for null opinions')

    for row in tqdm(original_data):
        data = {}
        if 't1' in file:
            if blank_cnt <= threshold and len(row['opinions'])==0: blank_cnt += 1
            elif len(row['opinions'])==0: continue

        row['text'] = START_WORD + row['text']
        data['sent_id'] = row['sent_id']
        data['raw_words'] = row['text']
        data['words'] = data['raw_words'].split()
        aspects, holders, opinions = [], [], []

        for cnt, original_opinion in enumerate(row['opinions']):
            aspect, holder, opinion = {}, {}, {}
            aspect['index'], holder['index'], opinion['index'] = cnt, cnt, cnt

            ############################################# Aspect #############################################
            try:aspect['from'], aspect['to'] = char2word_offset(data['raw_words'], original_opinion['Target'][1][0])
            except: aspect['from'], aspect['to'] = NULL_INDEX, NULL_INDEX+1
            if aspect['from'] == aspect['to']-1 == NULL_INDEX:
                aspect['term'] = NULL_TOKEN
            else:
                aspect['term'] = original_opinion['Target'][0][0]

            try: aspect['Polarity'] = str(original_opinion['Polarity'])
            except: aspect['Polarity'] = NULL_TOKEN; print(original_opinion)
            try: aspect['Intensity'] = str(original_opinion['Intensity'])
            except: aspect['Intensity'] = NULL_TOKEN; print(original_opinion)

            ############################################# Holder #############################################
            try:holder['from'], holder['to'] = char2word_offset(data['raw_words'], original_opinion['Source'][1][0])
            except: holder['from'], holder['to'] = NULL_INDEX, NULL_INDEX+1
            if holder['from'] == holder['to']-1 == NULL_INDEX:
                holder['term'] = NULL_TOKEN
            else:
                holder['term']  = original_opinion['Source'][0][0]

            ############################################# Opinion #############################################
            try:opinion['from'], opinion['to'] = char2word_offset(data['raw_words'], original_opinion['Polar_expression'][1][0])
            except: opinion['from'], opinion['to'] = NULL_INDEX, NULL_INDEX+1
            if opinion['from'] == opinion['to']-1 == NULL_INDEX:
                opinion['term'] = NULL_TOKEN
            else:
                opinion['term'] = original_opinion['Polar_expression'][0][0]

            aspects.append(aspect)
            holders.append(holder)
            opinions.append(opinion)

        if len(aspects) == 0:
            tmp = {}; tmp['index'] = 0; tmp['from']=NULL_INDEX; tmp['to']=NULL_INDEX+1
            for k in 'term Polarity Intensity'.split(): tmp[k]=NULL_TOKEN
            aspects = [tmp]
        if len(holders) == 0:
            tmp = {}; tmp['index'] = 0; tmp['from']=NULL_INDEX; tmp['to']=NULL_INDEX+1
            for k in 'term'.split(): tmp[k]=NULL_TOKEN
            holders = [tmp]
        if len(opinions) == 0:
            tmp = {}; tmp['index'] = 0; tmp['from']=NULL_INDEX; tmp['to']=NULL_INDEX+1
            for k in 'term'.split(): tmp[k]=NULL_TOKEN
            opinions = [tmp]

        data['aspects'] = aspects
        data['holder'] = holders
        data['opinions'] = opinions
        final_data.append(data)

    return final_data

In [ ]:
null_dict = {
    'mpqa': 0.1,
    'darmstadt_unis': 0.2,
    'norec': 0.3,
    'opener_en': 0,
    'multibooked_eu': 0.07,
    'multibooked_eu_crosslingual': 0.07,
    'multibooked_ca': 0.15,
    'opener_es': 0.06,
    "eng_combined": 0.1
}

for folder in os.listdir('../../final_data'):
    for file in os.listdir(f'../../final_data/{folder}'):
        if '1.' not in file: continue
        with open(f'../../final_data/{folder}/{file}', encoding="utf8") as f: data = json.load(f)
        print(f'../../final_data/{folder}/{file}')

        data = format_convert(data, file, null_dict[folder])
        print(f"salah di {file}")
        if 't1' in file: dest = 'train_convert.json'
        elif 'd1' in file: dest = 'dev_convert.json'
        else: dest = 'test_convert.json'

        with open(f'../../final_data/{folder}/{dest}', 'w') as f:
            json.dump(data, f)
    print()